In [2]:
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from pandas import DataFrame
from pandas.core.dtypes import common as com
from core.loader import Loader
from models.model_wrapper import ModelWrapper

from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TODO: Zkusit 3 stupně na NN, SVM a porovnat s LightGBM + XGBoost

# 4 x Malware
# 4x  Phishing
# + DGA (jeden stupen)

2025-04-27 23:55:12.821117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-27 23:55:12.821152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-27 23:55:12.850166: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 23:55:12.915963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 23:55:14.052879: W tensorflow/compiler/tf2


device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


2025-04-27 23:55:19.363243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-27 23:55:19.669383: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-27 23:55:19.672266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [1]:
!pip3 install tabulate

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pickle

def load_saved_split(stage, label, folder="./data/"):
    filename = ""
    if stage == 1:
        filename = f"validation_stage_1_{label}"
    elif stage == 2:
        filename = f"validation_stage_2_{label}"
    elif stage == 3:
        filename = f"validation_stage_3_{label}"
    else:
        raise ValueError("Invalid stage. Choose 1, 2, or 3.")
    with open(f'{folder}{filename}.pkl', "rb") as f:
        dump = pickle.load(f)
        X_test = dump["X_test"]
        y_test = dump["Y_test"]
    return X_test, y_test


In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, confusion_matrix
import os 
import joblib
from tabulate import tabulate

from sklearn.metrics import (
    f1_score, confusion_matrix, classification_report,
    accuracy_score, precision_score, recall_score, roc_auc_score
)


class ModelValidator:
    """
    A class for validating machine learning models with various metrics.

    Attributes:
        model (estimator): The machine learning model to be validated.
        X_test (array-like): The test dataset features.
        y_test (array-like): The true labels corresponding to X_test.
        arch_name (str): Optional architecture name for logging and file naming.
        label (str): Optional label (e.g., 'malware', 'phishing') for context.
        prefix (str): Optional prefix for versioning stages.
        version (str): Optional version identifier.
    """

    def __init__(self, model, X_test, y_test, arch_name=None, label=None, prefix=None, version=None):
        """
        Initializes the ModelValidator.

        Parameters:
           model (estimator): The trained machine learning model.
           X_test (array-like): Features from the test dataset.
           y_test (array-like): True labels for the test dataset.
           arch_name (str): Model architecture name (optional).
           label (str): Dataset label name (optional).
           prefix (str): Stage prefix (optional).
           version (str): Model version (optional).
        """
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        # New attributes for naming
        self.arch_name = arch_name
        self.label = label
        self.prefix = prefix
        self.version = version
        self.matrix_folder = 'confusion'
        self.tex_path = "./tex_sources/evaluation_metrics.tex"  # ← force the folder and path!

        # Create the folder immediately (safe even if it already exists)
        os.makedirs(os.path.dirname(self.tex_path), exist_ok=True)
    

    def evaluate_performance(self):
        """
        Evaluates and prints the model's performance metrics, including F1 score,
        confusion matrix, and a classification report. Also saves a confusion matrix plot.
        """
        print("\n🔍 Starting model evaluation...")
        
        def svm_convert(df):
            df = df.fillna(0)
            df = df.replace({True: 1, False: 0})
            return np.array(df)
                
                
        ### Variable preprocessing for different architectures and models ###
        if self.arch_name == 'feedforward':
            # progres emoji
            print(f"🔄 Detected Neural network, using: {self.label} scaler")
            if self.label == 'malware':
                self.X_test = joblib.load(f"scalers/malware_scaler.joblib").transform(self.X_test)
            elif self.label == 'phishing':
                self.X_test = joblib.load(f"scalers/phishing_scaler.joblib").transform(self.X_test)
        
        elif self.arch_name == 'svm':
            print(f"🔄 Detected SVM, using: {self.label} scaler")
            if self.label == 'malware':
                self.X_test = joblib.load(f"scalers/malware_scaler.joblib").transform(self.X_test)
            elif self.label == 'phishing':
                self.X_test = joblib.load(f"scalers/phishing_scaler.joblib").transform(self.X_test)
                
            self.X_test = svm_convert(self.X_test)

        
        y_pred = self.model.predict(self.X_test)

        # --- Safe prediction processing ---
        # Handle probability outputs (e.g., from neural networks)
        if np.issubdtype(y_pred.dtype, np.floating):
            if y_pred.ndim == 2 and y_pred.shape[1] > 1:
                # Multiclass or binary with probabilities -> argmax
                y_pred = np.argmax(y_pred, axis=1)
            else:
                # Single probability output -> threshold at 0.5
                y_pred = (y_pred >= 0.5).astype(int)

        f1 = f1_score(self.y_test, y_pred)
        conf_matrix = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = conf_matrix.ravel()

        fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
        tpr = tp / (tp + fn) if (tp + fn) != 0 else 0

        # --- Compute standard metrics ---
        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred, zero_division=0)
        recall = recall_score(self.y_test, y_pred, zero_division=0)

        # ROC AUC needs probability scores; fallback if not available
        try:
            if hasattr(self.model, "predict_proba"):
                y_proba = self.model.predict_proba(self.X_test)[:, 1]
            else:
                y_proba = y_pred  # fallback if no probabilities
            roc_auc = roc_auc_score(self.y_test, y_proba)
        except Exception:
            roc_auc = 0.0
            
                # --- Collect all metrics now ---
        metrics = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc,
            "True Negatives (TN)": tn,
            "False Positives (FP)": fp,
            "False Negatives (FN)": fn,
            "True Positives (TP)": tp,
            "False Positive Rate (FPR)": fpr,
            "True Positive Rate (TPR)": tpr,
        }
            
            
        # Pretty table print
        self.print_ascii_table(metrics)

        # Append LaTeX table
        self.append_latex_table(metrics)

        print("\n📝 Classification Report:")
        print(classification_report(self.y_test, y_pred, digits=6))

        # Plot confusion matrix
        self.plot_confusion_matrix(y_pred)
        
    def print_ascii_table(self, metrics_dict):
        """
        Prints a nicely formatted ASCII table of metrics.
        """
        headers = ["Metric", "Value"]
        rows = [(k, v) for k, v in metrics_dict.items()]
        table = tabulate(rows, headers=headers, tablefmt="grid", floatfmt=".4f")
        print("\n📋 Evaluation Summary:")
        print(table)

    def append_latex_table(self, metrics_dict):
        """
        Appends a polished LaTeX table of metrics to the specified .tex file.
        Creates the file if it doesn't exist.
        """
        # No need to create folder here anymore — already done in __init__

        # Custom mapping for Czech + English metric names
        metric_names = {
            "Accuracy": "Přesnost (Accuracy)",
            "F1 Score": "F1 Skóre",
            "Precision": "Precision (Přesnost)",
            "Recall": "Recall (Úplnost)",
            "ROC AUC": "ROC AUC",
            "True Negatives (TN)": "True Negatives (TN)",
            "False Positives (FP)": "False Positives (FP)",
            "False Negatives (FN)": "False Negatives (FN)",
            "True Positives (TP)": "True Positives (TP)",
            "False Positive Rate (FPR)": "False Positive Rate (FPR)",
            "True Positive Rate (TPR)": "True Positive Rate (TPR)",
        }

        table_rows = ""
        for k, v in metrics_dict.items():
            nice_name = metric_names.get(k, k)
            table_rows += f"{nice_name} & \\texttt{{{v:.4f}}} \\\\\n"

        section_name = f"{self.arch_name or 'Model'} {self.prefix or ''} {self.label or ''} {self.version or ''}".strip()

        latex_table = (
            f"\\section*{{Výsledky hodnocení: {section_name}}}\n"
            "\\begin{table}[h!]\n"
            "\\centering\n"
            "\\begin{tabular}{|l|c|}\n"
            "\\hline\n"
            "\\textbf{Metrika} & \\textbf{Hodnota} \\\\\n"
            "\\hline\n"
            f"{table_rows}"
            "\\hline\n"
            "\\end{tabular}\n"
            f"\\caption{{Výsledky klasifikace modelu {section_name}}}\n"
            f"\\label{{tab:{(self.label or 'model').lower()}_{(self.arch_name or 'model').lower()}}}\n"
            "\\end{table}\n\n"
        )

        with open(self.tex_path, "a") as f:
            f.write(latex_table)

        print(f"📄 Metrics table appended to {self.tex_path}")


    def plot_confusion_matrix(self, y_pred):
        # Attempt to get classes dynamically
        try:
            classes = self.model.classes_
        except AttributeError:
            classes = np.unique(self.y_test)

        cm = confusion_matrix(self.y_test, y_pred, labels=classes)
        plt.figure(figsize=(8, 6))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Reds)
        plt.title('Confusion Matrix', fontsize=18)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45, fontsize=14)
        plt.yticks(tick_marks, classes, fontsize=14)

        cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
        thresh = cm_norm.max() / 2.
        for i, j in np.ndindex(cm.shape):
            plt.text(j, i, f"{cm_norm[i, j]:.4f}\n({cm_norm[i, j]*100:.2f}%)",
                     horizontalalignment="center",
                     color="black" if cm_norm[i, j] > thresh else "black", fontsize=16)

        plt.tight_layout()
        plt.ylabel('True label', fontsize=14)
        plt.xlabel('Predicted label', fontsize=14)

        # Save the plot using your naming convention
        if all(v is not None for v in [self.arch_name, self.prefix, self.label, self.version]):
            filename = f"{self.arch_name}_{self.prefix}_{self.label}_{self.version}_confusion_matrix.png"
        else:
            filename = "confusion_matrix.png"

        os.makedirs(self.matrix_folder, exist_ok=True)
        save_path = os.path.join(self.matrix_folder, filename)
        plt.savefig(save_path, bbox_inches='tight')
        print(f"🖼️ Confusion matrix plot saved to: {save_path}\n")
        plt.close()


In [ ]:
model_wrapper = ModelWrapper(model_dir="models")
ARCH_NAME = "XgBoost" 
#ARCH_NAME = "Lgbm"
#ARCH_NAME = "feedforward"
VERSION = "v1.1"
#VERSION = "v1.0"
malicious_label = "malware"
stage = 3

# combination
# TODO: solve scalers for NN for each stage

model = model_wrapper.load(arch_name=ARCH_NAME, label=malicious_label, prefix=f"stage_{stage}", version=VERSION)
x_test, y_test = load_saved_split(3, malicious_label, folder="./data/")

validator = ModelValidator(model, x_test, y_test, arch_name=ARCH_NAME, label=malicious_label, prefix=f"stage_{stage}", version=VERSION)
validator.evaluate_performance()


📦 Loading model from models/XgBoost_stage_3_malware_v1.1.xgb

🔍 Starting model evaluation...

📋 Evaluation Summary:
+---------------------------+------------+
| Metric                    |      Value |
+===========================+============+
| True Negatives (TN)       | 81489.0000 |
+---------------------------+------------+
| False Positives (FP)      |   154.0000 |
+---------------------------+------------+
| False Negatives (FN)      |   357.0000 |
+---------------------------+------------+
| True Positives (TP)       |  9714.0000 |
+---------------------------+------------+
| False Positive Rate (FPR) |     0.0019 |
+---------------------------+------------+
| True Positive Rate (TPR)  |     0.9646 |
+---------------------------+------------+
| F1 Score                  |     0.9744 |
+---------------------------+------------+
📄 Metrics table appended to evaluation_metrics.tex

📝 Classification Report:
              precision    recall  f1-score   support

           0   0.9956

In [ ]:
# combination
architectures = ["feedforward", "XgBoost", "Lgbm", "svm"]
malicious_labels = ["malware", "phishing"]
stages = [1, 2, 3]

for architecture in architectures:       
    for malicious_label in malicious_labels:
        for stage in stages:
            model = model_wrapper.load(arch_name=architecture, label=malicious_label, prefix=f"stage_{stage}", version=VERSION)
            x_test, y_test = load_saved_split(stage, malicious_label)
            validator = ModelValidator(model, x_test, y_test, arch_name=architecture, label=malicious_label, prefix=f"stage_{stage}", version=VERSION)
            validator.evaluate_performance()

📦 Loading model from models/feedforward_stage_1_malware_v1.1.keras
🔄 Detected Neural network, using: malware scaler


/home/poli/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 62 features, but StandardScaler is expecting 176 features as input.